In [ ]:
from pynq import Overlay, MMIO
import time

In [2]:
overlay = Overlay('antminerS9I2S.bit')

In [ ]:
overlay?

In [3]:
fifo_mm_instance = overlay.ip_dict["axi_fifo_mm_s_0"]

In [14]:
# Base address and address range for the AXI interface
base_address = 0x43C00000  # Replace with your actual base address
address_range = 0x10000    # Replace with your actual address range (64KB)

# Create MMIO object to access the memory-mapped registers
mmio = MMIO(base_address, address_range)

# Define some register offsets (replace with actual addresses)
I2S_CONTROL_REG = 0x00  # Hypothetical offset for I2S control register
I2S_STATUS_REG = 0x04   # Hypothetical offset for I2S status register
I2S_FIFO_REG = 0x08     # Hypothetical offset for I2S FIFO register

In [15]:
# Function to configure I2S DAC
def configure_i2s():
    control_value = 0x01  # Example value to configure I2S (e.g., enable DAC)
    mmio.write(I2S_CONTROL_REG, control_value)  # Write to I2S control register
    print(f"Written {control_value} to I2S control register")

# Function to read the I2S status register
def read_i2s_status():
    status = mmio.read(I2S_STATUS_REG)  # Read from I2S status register
    print(f"I2S Status Register: {status}")
    return status

# Function to write audio data to I2S FIFO
def write_audio_data(data):
    mmio.write(I2S_FIFO_REG, data)  # Write audio data to I2S FIFO
    print(f"Written audio data {data} to I2S FIFO")

# Function to read audio data from I2S FIFO
def read_audio_data():
    data = mmio.read(I2S_FIFO_REG)  # Read audio data from I2S FIFO
    print(f"Read audio data: {data}")
    return data

In [ ]:
time.sleep(4)
print("Configuring I2S DAC...")
configure_i2s()
time.sleep(1)

In [ ]:
time.sleep(4)
print("Reading I2S status...")
read_i2s_status()

In [ ]:
time.sleep(4)

# Example: writing and reading audio data
audio_sample = 0x1234  # Replace with actual audio data
write_audio_data(audio_sample)

In [ ]:
read_audio_data()